PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=c340d1b08fc56e42ecc94c348a722a24dc565452bca9eaaf9d651aebdd214dbb
  Stored in directory: /root/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [ ]:
spark

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2025-11-24 05:16:05--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.242.34, 3.5.80.163, 52.92.206.114, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.242.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.07s   

2025-11-24 05:16:06 (524 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [ ]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Nov 20 14:30 sample_data
-rw-r--r-- 1 root root 36240 Jan 31  2021 boston_housing.csv


In [ ]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [ ]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [ ]:
data_2 = assembler.transform(data)

In [ ]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [ ]:
train, test = data_2.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

## 모델 성능 측정

In [ ]:
evaluation_summary = model.evaluate(test)

In [ ]:
evaluation_summary

In [ ]:
evaluation_summary.meanAbsoluteError

3.091402850413186

In [ ]:
evaluation_summary.rootMeanSquaredError

4.176746857068049

In [ ]:
evaluation_summary.r2

0.7633974030355386

## 모델 예측값 살펴보기

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0| 0.389|6.453|31.9| 7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|27.383456889617687|
|0.01311|90.0| 1.22|   0| 0.403|7.249|21.9| 8.6966|  5|226|   17.9|395.93| 4.81|35.4|[0.01311,90.0,1.2...|30.908243021786113|
| 0.0136|75.0|  4.0|   0|  0.41|5.888|47.6| 7.3197|  3|469|   21.1| 396.9| 14.8|18.9|[0.0136,75.0,4.0,...|15.797455505215112|
|0.01439|60.0| 2.93|   0| 0.401|6.604|18.8| 6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...| 31.68441622976716|
|0.01538|90.0| 3.75|   0| 0.394|7.454|34.2| 6.3361|  3|244|   15.9|386.34| 3.11|44.0|[0.01538,90.0,3.7...| 37.49167149

In [ ]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|22.0|[0.01096,55.0,2.2...|27.383456889617687|
|35.4|[0.01311,90.0,1.2...|30.908243021786113|
|18.9|[0.0136,75.0,4.0,...|15.797455505215112|
|29.1|[0.01439,60.0,2.9...| 31.68441622976716|
|44.0|[0.01538,90.0,3.7...| 37.49167149734494|
|30.1|[0.01709,90.0,2.0...| 24.54007002368853|
|42.3|[0.02177,82.5,2.0...| 37.22974642893032|
|23.9|[0.02543,55.0,3.7...|  27.8035036500265|
|26.6|[0.02899,40.0,1.2...|22.232446642324234|
|18.5|[0.03041,0.0,5.19...|19.163481508117435|
|31.2|[0.03049,55.0,3.7...|  28.6292763052286|
|34.9|[0.0315,95.0,1.47...|30.330020628996017|
|33.4|[0.03237,0.0,2.18...|28.712379215546022|
|24.1|[0.03445,82.5,2.0...|29.390412847329355|
|19.4|[0.03466,35.0,6.0...|23.381759074951496|
|22.0|[0.03537,34.0,6.0...|29.034719354903245|
|23.5|[0.03584,80.0,3.3...|30.550458308277005|
|24.8|[0.03659,25.0,4.8...|26.342472463523816|
|35.4|[0.0370

In [ ]:
model.save("boston_housing_model")

In [ ]:
!ls boston_housing_model

data  metadata


In [ ]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Nov 24 05:33 data
drwxr-xr-x 2 root root 4096 Nov 24 05:33 metadata


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model"
model.save(path)

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [ ]:
predictions2 = loaded_model.transform(test)

In [ ]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|22.0|[0.01096,55.0,2.2...|27.383456889617687|
|35.4|[0.01311,90.0,1.2...|30.908243021786113|
|18.9|[0.0136,75.0,4.0,...|15.797455505215112|
|29.1|[0.01439,60.0,2.9...| 31.68441622976716|
|44.0|[0.01538,90.0,3.7...| 37.49167149734494|
|30.1|[0.01709,90.0,2.0...| 24.54007002368853|
|42.3|[0.02177,82.5,2.0...| 37.22974642893032|
|23.9|[0.02543,55.0,3.7...|  27.8035036500265|
|26.6|[0.02899,40.0,1.2...|22.232446642324234|
|18.5|[0.03041,0.0,5.19...|19.163481508117435|
|31.2|[0.03049,55.0,3.7...|  28.6292763052286|
|34.9|[0.0315,95.0,1.47...|30.330020628996017|
|33.4|[0.03237,0.0,2.18...|28.712379215546022|
|24.1|[0.03445,82.5,2.0...|29.390412847329355|
|19.4|[0.03466,35.0,6.0...|23.381759074951496|
|22.0|[0.03537,34.0,6.0...|29.034719354903245|
|23.5|[0.03584,80.0,3.3...|30.550458308277005|
|24.8|[0.03659,25.0,4.8...|26.342472463523816|
|35.4|[0.0370